In [5]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

In [7]:
DATA_PATH = r"D:\sythetic_data\dataset_100\256x256"

In [158]:

def Traver(mode,DATA_PATH):
    data=[]
    print(f"######explore {mode} dataset######")
    path = os.path.join(DATA_PATH, mode, "labels")
    file = os.listdir(path)
    file_num = 1
    for i in tqdm(file, desc=f"{file_num}/{len(file)}", unit= "i" ) : 
        mask = np.array(Image.open(os.path.join(path, i)))
        cls, counts = np.unique(mask, return_counts=True) 
        if (cls == 18).any():
            index = np.where(cls == 18) 
            num = counts[index[0]][0]
            ID = i.split('_seg')[0] 
            data.append([ID, cls[index[0]][0], num])
        file_num += 1
    return data

In [160]:
mode = 'train'
facewear_train = Traver(mode, DATA_PATH)

######explore train dataset######


1/60000: 100%|████████████████████████████████████████████████████████████████████| 60000/60000 [24:56<00:00, 40.10i/s]


In [162]:
facewear_train

[]

In [164]:
mode = 'val'
facewear_val = Traver(mode, DATA_PATH)

######explore val dataset######


1/20000: 100%|████████████████████████████████████████████████████████████████████| 20000/20000 [08:40<00:00, 38.44i/s]


In [166]:
facewear_val

[]

In [168]:
mode = 'test'
facewear_test = Traver(mode, DATA_PATH)

######explore test dataset######


1/20000: 100%|████████████████████████████████████████████████████████████████████| 20000/20000 [08:56<00:00, 37.29i/s]


In [170]:
facewear_test

[]

In [172]:
df = pd.DataFrame(facewear_train, columns=['id', 'cls', 'num'])
print(df)

Empty DataFrame
Columns: [id, cls, num]
Index: []


## Remapping

In [12]:

"""Remap mask which smplified classes"""
class_remapping = {
     0: [0],
     1: [1,2],
     2: [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
   255: [255],
 }


In [13]:
reverse_mapping = {}
for new_class, old_classes in class_remapping.items():
    for old_class in old_classes:
        reverse_mapping[old_class] = new_class

In [14]:
mode = "train" #"val", "test"

In [15]:
input_folder = os.path.join(DATA_PATH, mode, "labels") 
output_folder = os.path.join(DATA_PATH, mode, "new_labels")  

os.makedirs(output_folder, exist_ok=True)

In [16]:
for filename in os.listdir(input_folder):
    input_path = os.path.join(input_folder, filename)

    if os.path.isfile(input_path) and filename.endswith('.png'): 
    
        img = Image.open(input_path)
        img = np.array(img)  
        
        remapped_img = np.vectorize(lambda x: reverse_mapping.get(x, x))(img)
        remapped_img = Image.fromarray(remapped_img.astype(np.uint8))
        output_path = os.path.join(output_folder, filename)
        remapped_img.save(output_path)


In [19]:
image = np.array(Image.open(os.path.join(input_folder, "000001_seg.png")))

In [20]:
np.unique(image)

array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 11, 12, 13, 15], dtype=uint8)

In [21]:
image = np.array(Image.open(os.path.join(output_folder, "000001_seg.png")))
np.unique(image)

array([0, 1, 2], dtype=uint8)